# Лабораторная работа №5
## Выполнил студент группы Балыко Владислав Васильевич БФИ2201

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [5]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [11]:
def kmp_search(text, pattern):
    n = len(text)
    m = len(pattern)
    
    # Препроцессинг паттерна для создания lps массива
    lps = [0] * m
    j = 0  # длина предыдущего совпадения префикса и суффикса
    i = 1
    
    while i < m:
        if pattern[i] == pattern[j]:
            j += 1
            lps[i] = j
            i += 1
        else:
            if j != 0:
                j = lps[j - 1]
            else:
                lps[i] = 0
                i += 1
    
    i = 0  # индекс для текста
    j = 0  # индекс для паттерна
    
    while i < n:
        if pattern[j] == text[i]:
            i += 1
            j += 1
        
        if j == m:
            return i - j  # Найдено совпадение
        
        elif i < n and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1
    
    return -1  # Совпадение не найдено


def measure_time(func, text, pattern):
    start = t.now()
    result = func(text, pattern)
    end = t.now()
    return result, (end - start).total_seconds()


text = input("Введите строку: ")
pattern = input("Введите подстроку: ")
case_sensitive = input("Чувствительность к регистру? (y/n): ").lower() == 'y'
    
if not case_sensitive:
    text = text.lower()
    pattern = pattern.lower()
    
# Стандартный поиск
standard_result, standard_time = measure_time(lambda t, p: t.find(p), text, pattern)
print(f"Стандартная функция поиска: индекс {standard_result}, время {standard_time:.6f} сек")
    
# Алгоритм Кнута-Морриса-Пратта
kmp_result, kmp_time = measure_time(kmp_search, text, pattern)
print(f"Алгоритм Кнута-Морриса-Пратта: индекс {kmp_result}, время {kmp_time:.6f} сек")

Стандартная функция поиска: индекс 568, время 0.000000 сек
Алгоритм Кнута-Морриса-Пратта: индекс 568, время 0.000000 сек


#### Упрощенный алгоритм Бойера-Мура

In [10]:
def boyer_moore_search(text, pattern):
    n = len(text)
    m = len(pattern)
    
    if m == 0:
        return -1
    
    # Таблица плохих символов
    bad_char = defaultdict(lambda: -1)
    for i in range(m):
        bad_char[pattern[i]] = i
    
    s = 0  # смещение для текста
    while s <= n - m:
        j = m - 1
        
        while j >= 0 and pattern[j] == text[s + j]:
            j -= 1
        
        if j < 0:
            return s
        else:
            s += max(1, j - bad_char[text[s + j]])
    
    return -1  # Совпадение не найдено


def measure_time(func, text, pattern):
    start = t.now()
    result = func(text, pattern)
    end = t.now()
    return result, (end - start).total_seconds()



text = input("Введите строку: ")
pattern = input("Введите подстроку: ")
case_sensitive = input("Чувствительность к регистру? (y/n): ").lower() == 'y'
    
if not case_sensitive:
    text = text.lower()
    pattern = pattern.lower()
    
# Стандартный поиск
standard_result, standard_time = measure_time(lambda t, p: t.find(p), text, pattern)
print(f"Стандартная функция поиска: индекс {standard_result}, время {standard_time:.6f} сек")
    
# Упрощенный алгоритм Бойера-Мура
bm_result, bm_time = measure_time(boyer_moore_search, text, pattern)
print(f"Упрощенный алгоритм Бойера-Мура: индекс {bm_result}, время {bm_time:.6f} сек")

Стандартная функция поиска: индекс 568, время 0.000000 сек
Упрощенный алгоритм Бойера-Мура: индекс 568, время 0.000000 сек


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [15]:
import math
from queue import PriorityQueue

N = 4  # Размерность игры
SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 4, 
         5, 6, 7, 8, 
         9, 10, 11, 12, 
         13, 0, 14, 15]

# Проверка решаемости позиции
def is_solvable(puzzle):
    inversions = 0
    puzzle_len = len(puzzle)
    puzzle_side = int(puzzle_len ** 0.5)

    for i in range(puzzle_len):
        for j in range(i + 1, puzzle_len):
            if puzzle[i] != 0 and puzzle[j] != 0 and puzzle[i] > puzzle[j]:
                inversions += 1

    zero_row = puzzle.index(0) // puzzle_side

    if puzzle_side % 2 == 1:
        return inversions % 2 == 0
    else:
        return (inversions + zero_row) % 2 == 1

# Генерация возможных ходов
def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)     # Вверх
    if i < N - 1: offsets.append(N)  # Вниз
    if j > 0: offsets.append(-1)     # Влево
    if j < N - 1: offsets.append(1)  # Вправо
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x == blank else position[blank] if x == swap else position[x] for x in range(N * N))

# Вычисление эвристики Манхэттена
def manhattan(position):
    distance = 0
    for i, value in enumerate(position):
        if value == 0:
            continue
        target_row, target_col = divmod(value - 1, N)
        current_row, current_col = divmod(i, N)
        distance += abs(target_row - current_row) + abs(target_col - current_col)
    return distance

# Класс позиции для A*
class Position:
    def __init__(self, position, g, h):
        self.position = position
        self.g = g  # Дистанция от начальной позиции
        self.h = h  # Эвристическая оценка до цели

    def __lt__(self, other):
        return (self.g + self.h) < (other.g + other.h)

# Проверка решаемости и нахождение решения
if not is_solvable(start):
    print('Нерешаемая позиция')
else:
    start = tuple(start)
    p = Position(start, 0, manhattan(start))
    candidates = PriorityQueue()
    candidates.put(p)
    visited = set([p.position])
    came_from = {p.position: None}
    
    while p.position != SOLVED:
        p = candidates.get()
        for neighbor in moves(p.position):
            if neighbor not in visited:
                g = p.g + 1
                h = manhattan(neighbor)
                new_position = Position(neighbor, g, h)
                candidates.put(new_position)
                came_from[neighbor] = p
                visited.add(neighbor)
    
    # Построение пути
    path = []
    current = p
    while current.position != start:
        prev = came_from[current.position]
        blank_index = prev.position.index(0)
        moved_tile = current.position[blank_index]
        path.append(moved_tile)
        current = prev
    path.reverse()

    print(path)


[14, 15]


### Вывод